# MobileNet Models

### AGP Pruning 


https://nervanasystems.github.io/distiller/model_zoo.html

Checkpoints:

https://s3-us-west-1.amazonaws.com/nndistiller/agp-pruning/mobilenet/checkpoint.pth.tar

BaseLine:

https://github.com/marvis/pytorch-mobilenet


But I would need to retrain

In [1]:
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [11]:
# Load some common jupyter code
%run './distiller_jupyter_helpers.ipynb'
from distiller.models import create_model
from distiller.apputils import *
import qgrid

from ipywidgets import *
from bqplot import *
import bqplot.pyplot as bqplt
from functools import partial

import weightwatcher as ww
import torchvision.models as models


import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

marker_size = 75
plt.rcParams['figure.figsize'] = [5,5]
matplotlib.rcParams.update({'font.size': 20})
legend_prop={'size':16}

In [3]:
!rm checkpoint.pth.tar 
!wget https://s3-us-west-1.amazonaws.com/nndistiller/agp-pruning/mobilenet/checkpoint.pth.tar 
!mv checkpoint.pth.tar  mobilenet_finetuned.pth.tar 


rm: checkpoint.pth.tar: No such file or directory
--2019-04-17 21:02:39--  https://s3-us-west-1.amazonaws.com/nndistiller/agp-pruning/mobilenet/checkpoint.pth.tar
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.28.25
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.28.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50527563 (48M) [application/x-tar]
Saving to: ‘checkpoint.pth.tar’

checkpoint.pth.tar  100%[===================>]  48.19M  8.14MB/s    in 7.8s    

2019-04-17 21:02:47 (6.20 MB/s) - ‘checkpoint.pth.tar’ saved [50527563/50527563]



In [4]:
!ls mobilenet*tar

mobilenet_finetuned.pth.tar


In [8]:
cpfiles = {
    'mobilenet_finetuned.pth.tar': 66.24
}

In [9]:
results = {}

for file, accuracy in cpfiles.items():
    logger.info(file)
    checkpoint_file = file

    try:
        mobilenet_model = create_model(False, 'imagenet', 'mobilenet', parallel=True)
        load_checkpoint(mobilenet_model, checkpoint_file);
        watcher = ww.WeightWatcher(model=mobilenet_model, logger=logger)
        watcher.analyze(compute_alphas=True)
        summary = watcher.get_summary()
        summary['accuracy'] = accuracy
        results[file] = summary
    except Exception as e:
        print("Did you forget to download the checkpoint file?")
        raise e

INFO:__main__:mobilenet_finetuned.pth.tar
INFO:root:=> using mobilenet model for ImageNet
INFO:root:=> loading checkpoint mobilenet_finetuned.pth.tar
INFO:root:=> Checkpoint contents:
╒═══════════════════╤═════════════╤═══════════════════╕
│ Key               │ Type        │ Value             │
╞═══════════════════╪═════════════╪═══════════════════╡
│ arch              │ str         │ mobilenet         │
├───────────────────┼─────────────┼───────────────────┤
│ best_top1         │ float       │ 66.24075056979612 │
├───────────────────┼─────────────┼───────────────────┤
│ compression_sched │ dict        │                   │
├───────────────────┼─────────────┼───────────────────┤
│ epoch             │ int         │ 199               │
├───────────────────┼─────────────┼───────────────────┤
│ optimizer         │ dict        │                   │
├───────────────────┼─────────────┼───────────────────┤
│ state_dict        │ OrderedDict │                   │
╘═══════════════════╧═══════════